In [ ]:
import pandas as pd
import numpy as np
import ast

In [ ]:
df = pd.read_csv("./data/movies_metadata.csv")
keywords_df = pd.read_csv("./data/keywords.csv")
credits_df = pd.read_csv("./data/credits.csv")

In [ ]:
del df["homepage"]
del df["video"]
df = df.drop_duplicates(subset=["id"], keep="first")
df = df.drop([35587, 19730, 29503, 19729, 29502, 35586]).reset_index(drop=True)
df = df.rename(columns={"original_language": "language", "release_date": "releaseDate", 
                        "adult": "isAdult", "runtime": "runtimeMinutes", 
                        "vote_average": "averageRating", "vote_count": "numVotes"})
df["isAdult"] = pd.factorize(df["isAdult"])[0]
df["id"] = df["id"].astype(int)

In [ ]:
keywords_df = keywords_df.drop_duplicates(subset=["id"], keep="first")
df = df.merge(keywords_df, how='left', on='id')

In [ ]:
credits_df = credits_df.drop_duplicates(subset=["id"], keep="first")
df = df.merge(credits_df, how='left', on='id')

In [ ]:
df.info()

In [ ]:
# df.to_csv("./data/new_movies_metadata.csv", index=False)

In [ ]:
basics = pd.read_csv("./title.basics.tsv/data.tsv", sep="\t", header=0)
basics = basics.drop_duplicates(subset=["tconst"], keep="first")
basics = basics[basics["tconst"].isin(df["imdb_id"].dropna())]
start_year = basics.set_index("tconst")["startYear"]
new_basics = pd.concat([df.dropna(subset=["imdb_id"]).set_index("imdb_id"), start_year], axis=1)
new_basics = pd.concat([new_basics.reset_index(names=["imdb_id"]), df[df["imdb_id"].isna()]])

In [ ]:
# new_basics.to_csv("./new_basics.csv", index=False)

In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
df = pd.read_csv("./data/new_movies_metadata.csv")
dic_columns = ["genres", "production_companies", "production_countries", "spoken_languages",
               "keywords", "cast", "crew"]
for c in dic_columns:
    df[c] = df[c].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) and x != '[]' else [])

In [11]:
unique_casts = set()
unique_actors = set()
for casts_list in df["cast"]:
  for cast in casts_list:
      unique_casts.add((cast.get("cast_id"), cast.get("character"), cast.get("credit_id"), cast.get("order")))
      unique_actors.add((cast.get("id"), cast.get("name"), cast.get("gender")))

In [18]:
unique_crews = set()
for crews_list in df["crew"]:
  for crew in crews_list:
      unique_crews.add((crew.get("id"), crew.get("job"), crew.get("department"), crew.get("credit_id"), crew.get("gender"), crew.get("name")))